# Keras Tuner Test

Kear Tuner has categories from Basic **RandomSearch** to subclass **HyperModel**. 
It takes an argument **hp** from which you can sample hyperparameters, such as **hp.Int('units', min_value=32, max_value=512, step=32)** (an integer from a certain range)

```python
# Define a model
def build_model(hp):
    ...
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu'))
    ...
    ...


# Use random search
from kerastuner.tuners import RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

# print a summary of the search space
tuner.search_space_summary()

# start to search the best set of parameters
tuner.search(x, y,
             epochs=5,
             validation_data=(val_x, val_y))

# retrieve the best model
models = tuner.get_best_models(num_models=2)

# print a summary of the results
tuner.results_summary()
```

Use a HyperModel subclass instead of a model-building function. This makes it easy to share and reuse hypermodels.


```python
from kerastuner import HyperModel

class MyHyperModel(HyperModel):
    def __init__(self, num_classes):
        
    def build(self, hp):
        ...
        
    
hypermodel = MyHyperModel(num_classes=10)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x, y,
             epochs=5,
             validation_data=(val_x, val_y))

```
    

Keras Tuner includes pre-made tunable applications: HyperResNet and HyperXception and You can easily restrict the search space to just a few parameters.

## Keras Tuner Mnist Tutorial

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy

from kerastuner.tuners import RandomSearch
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters


(x, y), (val_x, val_y) = keras.datasets.mnist.load_data()
x = x.astype('float32') / 255.
val_x = val_x.astype('float32') / 255.

x = x[:10000]
y = y[:10000]

def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='test_dir')

tuner.search_space_summary()

tuner.search(x=x,
             y=y,
             epochs=3,
             validation_data=(val_x, val_y))

tuner.results_summary()

In [ ]:
## use HyperModel

class MyHyperModel(HyperModel):

    def __init__(self, img_size, num_classes):
        self.img_size = img_size
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten(input_shape=self.img_size))
        for i in range(hp.Int('num_layers', 2, 20)):
            model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32),
                                   activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
        return model

    
# Define search parameters
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=False,
    objective='val_accuracy')

tuner.search(x=x,
             y=y,
             epochs=5,
             validation_data=(val_x, val_y))

## Keras Tuner HyperResNet on CIFAR 10 

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from kerastuner.applications import HyperResNet
from kerastuner import RandomSearch

# Import the Cifar10 dataset.
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)



In [ ]:
# Import an hypertunable version of Resnet.
hypermodel = HyperResNet(
    input_shape=x_train.shape[1:],
    classes=NUM_CLASSES)


In [ ]:
# Initialize the hypertuner: we should find the model that maximixes the
# validation accuracy, using 40 trials in total.
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=40,
    project_name='cifar10_resnet',
    directory='test_directory')

In [ ]:
# Display search overview.
tuner.search_space_summary()

# Performs the hypertuning.
tuner.search(x_train, y_train, epochs=10, validation_split=0.1)



In [ ]:
# Show the best models, their hyperparameters, and the resulting metrics.
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# Evaluate the best model.
loss, accuracy = best_model.evaluate(x_test, y_test)
print('loss:', loss)
print('accuracy:', accuracy)